In [39]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By # 웹사이트의 구성요소를 선택하기 위해 By 모듈을 불려옵니다
from selenium.webdriver.support.ui import WebDriverWait # 웹페이지가 전부 로드될때까지 기다리는 (Explicitly wait) 기능을 하는 모듈입니다
from webdriver_manager.chrome import ChromeDriverManager # 크롬에서 크롤링을 하기 위해, 웹 드라이버를 설치하는 모듈입니다
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support import expected_conditions as EC # 크롬의 어떤 부분의 상태를 확인하는 모듈입니다
import time # 정해진 시간만큼 기다리게 하기 위한 패키지입니다
from selenium.webdriver.common.keys import Keys

In [40]:
Tsherpa_url= "https://edu.tsherpa.co.kr/LectureInfo/LectureReviews2"


In [41]:
driver = webdriver.Chrome()
driver.get(Tsherpa_url)

In [42]:
try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="mainForm"]'))
    )
finally:
    pass

In [43]:
fnm = '' # 맨 첫번째 상호 이름입니다. 다음 페이지에서도 똑같은 상호가 나온다면, 다음 페이지가 없다고 인식하고, brk 값이 바뀌면서 반복문이 종료되지요
brk = 0
nxt = 2
res = [] # 결과 파일은 판다스 데이터프레임으로 입력할겁니다
page_count = 0
stop_code = 0

while True:
    
    page_count += 1
    brk += 1

    for i in range(1,11):
        
        time.sleep(0.5)
        hour = driver.find_element(by='xpath',value = f'//*[@id="mainForm"]/table/tbody/tr[{i}]/td[2]/a/span').text
        
        if driver.find_element(By.XPATH, f'//*[@id="mainForm"]/table/tbody/tr[{i}]/td[2]/a').is_displayed():
            driver.find_element(By.XPATH, f'//*[@id="mainForm"]/table/tbody/tr[{i}]/td[2]/a').send_keys(Keys.ENTER)
            time.sleep(1)
            driver.switch_to.window(driver.window_handles[-1])

            nm = driver.find_elements(By.XPATH, '//*[@id="mainForm"]/div[1]/span/a')
            yr = driver.find_elements(By.XPATH, '//*[@id="mainForm"]/div[2]/table/tbody/tr[2]/td/span[2]')
            scp = driver.find_elements(By.XPATH, '//*[@id="mainForm"]/div[2]/table/tbody/tr[2]/td/span[1]')
            desc = driver.find_elements(By.XPATH, '//*[@id="mainForm"]/div[2]/table/tbody/tr[3]/td')

        year = yr[0].text.split('|')[1].strip()
        year = year.split('.')[0].strip()
        print(year, type(year))
        if int(year) < 2018:
            stop_code += 1

        res.append([nm[0].text, hour, yr[0].text.split('|')[1].strip(), scp[0].text, desc[0].text])

        driver.close()
        driver.switch_to.window(driver.window_handles[0])


    
    # 다음 페이지로 넘어가는 코드입니다    
    if page_count != 11:
        driver.find_element(By.XPATH, f'//*[@id="mainForm"]/div[3]/a[{page_count+nxt}]').send_keys(Keys.ENTER)

    else:
       driver.find_element(By.XPATH, f'//*[@id="mainForm"]/div[3]/a[12]').send_keys(Keys.ENTER)
       page_count = 0

    time.sleep(2.5)
    
    # 코드 멈추기
    if stop_code == 1:
        print(page_count, brk)
        driver.quit()
        break


            





NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="mainForm"]/table/tbody/tr[10]/td[2]/a/span"}
  (Session info: chrome=118.0.5993.89); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7EF158EF2+54786]
	(No symbol) [0x00007FF7EF0C5612]
	(No symbol) [0x00007FF7EEF7A64B]
	(No symbol) [0x00007FF7EEFBB79C]
	(No symbol) [0x00007FF7EEFBB91C]
	(No symbol) [0x00007FF7EEFF6D87]
	(No symbol) [0x00007FF7EEFDBEAF]
	(No symbol) [0x00007FF7EEFF4D02]
	(No symbol) [0x00007FF7EEFDBC43]
	(No symbol) [0x00007FF7EEFB0941]
	(No symbol) [0x00007FF7EEFB1B84]
	GetHandleVerifier [0x00007FF7EF4A7F52+3524194]
	GetHandleVerifier [0x00007FF7EF4FD800+3874576]
	GetHandleVerifier [0x00007FF7EF4F5D7F+3843215]
	GetHandleVerifier [0x00007FF7EF1F5086+694166]
	(No symbol) [0x00007FF7EF0D0A88]
	(No symbol) [0x00007FF7EF0CCA94

In [44]:
df = pd.DataFrame(res)
df.to_csv('./res_Tsherpa.csv', index=False) # 데이터가 실시간으로 저장되도록 합니다
df

,0,1,2,3,4
0,파워포인트로 끝내는 교실 콘텐츠 창작 비법 노트,15시간,2023.10.23,,파워포인트로 이렇게 많은 것을 할 수 있다는 것을 새롭게 알게 되어서 많이 부끄럽습...
1,손그림과 파워포인트로 교실 속 콘텐츠 만들기,30시간,2023.10.23,,파워포인트 주로 사용하는데 매일 하던거만 하다 새로운걸 알아가네요~\n감사합니다~
2,쉽게 배우고 활용하는 교실 속 손그림 그리기,15시간,2023.10.22,,도형을 이용해 간단하게 손그림 그리는 방법을 배우는 유익한 연수였어요..\n그런데 ...
3,왕초보를 위한 캘리그래피,15시간,2023.10.19,,저는 미술에 정말 소질이 없는 사람입니다.. 그래서 아이들과 미술을 하는 것도 힘들...
4,같이 읽고 함께 나누는 독서수업방법,30시간,2023.10.19,,교직 생활이 연수가 늘어갈수록 다양한 독서 연수 강좌를 들어왔습니다. 이번 연수는 ...
...,...,...,...,...,...
1634,교과수업에 활용하는 진로교육,30시간,2017.12.30,,학교 진로교육의 목표 (2015개정)학생 자신의 진로를 창의적으로 개발하고 지속적으...
1635,스토리텔링을 활용한 초등수학 수업,30시간,2017.12.30,,"선생님들의 수학 개념 설명을 듣고, 실제 수학 수업을 볼 수 있어서 너무 유익했습니..."
1636,교과수업에 활용하는 진로교육,30시간,2017.12.29,,진로라는게 흥미는 있지만 너무 포괄적이네요.
1637,교과수업에 활용하는 진로교육,30시간,2017.12.26,,진로교육에 관해 많은 것을 알게 되었습니다.\n감사합니다.
